In [1]:
# from functions import get_current_location
# location = get_current_location()
# if location:
#     latitude, longitude = location
#     print(f'Current Latitude: {latitude}, Longitude: {longitude}')
# else:
#     print('Failed to determine current location.')

In [2]:
from route import Route
from route_api import RouteAPI

current_position = {'longitude': 130.841366,
                    'latitude': -12.45519}
final_position = {'longitude': 133.8750197,
                  'latitude': -23.7090616}
delta_spacing = 2000.0 # in meters
number_sites = 15

route = Route()
api_route = RouteAPI(route)

api_route_df = api_route.cut_route_data(current_position, number_sites=number_sites, print_is_requested=False)
# api_route_df = api_route.cut_route_data(current_position, number_sites=number_sites, delta_spacing=delta_spacing, print_is_requested=True)
# api_route_df = api_route.cut_route_data(current_position=current_position, final_position=final_position, number_sites=number_sites, print_is_requested=False)
# route.find_closest_row(current_position)
# route.find_closest_rows(route.get_control_stops_data)
# route.insert_to_control_stops()

In [3]:
from Solcast_executer import SolcastExecuter
solcast_executer = SolcastExecuter()

# solcast_executer.get_forecast(current_position)
raw_forecast_solcast = solcast_executer.get_forecasts(api_route_df)
print(raw_forecast_solcast)

                                        tt  gh     ff   dd    rh
cumDistance  time                                               
1.358064e+03 2023-10-12 11:45:00+00:00  28   0   8.28  263  69.3
             2023-10-12 12:00:00+00:00  28   0   8.64  265  70.0
             2023-10-12 12:15:00+00:00  28   0   9.36  266  70.6
             2023-10-12 12:30:00+00:00  28   0   9.72  268  71.2
             2023-10-12 12:45:00+00:00  28   0  10.08  270  71.7
...                                     ..  ..    ...  ...   ...
3.026918e+06 2023-10-14 10:45:00+00:00  12   0  12.96  210  83.9
             2023-10-14 11:00:00+00:00  12   0  12.24  206  84.0
             2023-10-14 11:15:00+00:00  12   0  11.52  202  84.1
             2023-10-14 11:30:00+00:00  12   0  10.80  198  84.1
             2023-10-14 11:45:00+00:00  11   0  10.08  194  84.0

[2895 rows x 5 columns]


In [4]:
from Meteotest_executer import MeteotestExecuter
from Meteotest_requester import MeteotestRequester
from Meteotest_parser import MeteotestParser

api_parser = MeteotestParser()
api_requester = MeteotestRequester(parser=api_parser, print_is_requested=True)
api_executer = MeteotestExecuter(requester=api_requester)

Response status from siteinfo: OK.
Current sites' info has been retrieved:
        name  longitude  latitude  altitude      UTC_offset
site_id                                                    
589020     0    130.845  -12.4560        32 0 days 09:30:00
589021     1    130.917  -12.4456        29 0 days 09:30:00
589022     2    131.001  -12.4775        26 0 days 09:30:00
589023     3    131.061  -12.5429        36 0 days 09:30:00
589024     4    131.073  -12.6294       102 0 days 09:30:00
...      ...        ...       ...       ...             ...
589165   145    133.809  -23.3695        15 0 days 09:30:00
589166   146    133.838  -23.4560        17 0 days 09:30:00
589167   147    133.866  -23.5425        14 0 days 09:30:00
589168   148    133.870  -23.6284        11 0 days 09:30:00
589169   149    133.874  -23.7081        52 0 days 09:30:00

[150 rows x 5 columns]


In [5]:
# api_requester.get_site_info(print_is_requested=True)
# api_executer.get_all_site_id

In [6]:
# api_requester.get_site_add(name="0", position=current_position, print_is_requested=True)
# api_executer.add_sites(api_route_df, print_is_requested=True)

In [7]:
# api_requester.get_site_delete(585394, print_is_requested=True)
# api_executer.delete_all_sites()

In [8]:
# site_id = 54689
# name = "404"
# position = {'latitude': 136,
#             'longitude': -30}

# api_requester.get_site_edit(site_id, name=name, position=position)

# api_route_df.index=api_executer.get_all_site_id ########TODO ADAPT DIRECTLY 
# api_executer.edit_sites(api_route_df)

In [9]:
# raw_df = api_requester.get_solar_forecast()

# raw_df = api_requester.get_solar_forecast_cloudmove()
# print(raw_df)
# raw_SF_df, _, raw_CM_df, _ = api_executer.get_new_forecasts()

# print(raw_SF_df.head(), raw_CM_df.head())

#TODO CHANGE TO 09.30 SE 10.30

In [10]:
sites_df = api_requester.get_current_sites

from preprocessor import Preprocessor
preprocessor = Preprocessor()
hours_in_advance = None
preprocessed_df, preprocessed_df = preprocessor.preprocess(api_route_df, sites_df, raw_forecast_solcast, hours_in_advance)
# preprocessor.save_data2folder()
print(preprocessed_df)

                                        temperature  globalIrradiance  \
cumDistance  time                                                       
1.358064e+03 2023-10-12 11:45:00+00:00           31                 0   
             2023-10-12 12:00:00+00:00           31                 0   
             2023-10-12 12:15:00+00:00           31                 0   
             2023-10-12 12:30:00+00:00           31                 0   
             2023-10-12 12:45:00+00:00           31                 0   
...                                             ...               ...   
3.026918e+06 2023-10-14 10:45:00+00:00           15                 0   
             2023-10-14 11:00:00+00:00           15                 0   
             2023-10-14 11:15:00+00:00           15                 0   
             2023-10-14 11:30:00+00:00           15                 0   
             2023-10-14 11:45:00+00:00           14                 0   

                                        windSpeed 

In [11]:
# import folium

# # Assuming you have these dataframes already
# control_stops_df = route.get_control_stops_data
# sites_df = api_requester.get_current_sites
# rt = route.get_route_data

# # Define the bounds
# min_lng = rt['longitude'].min()
# min_lat = rt['latitude'].min()
# max_lng = rt['longitude'].max()
# max_lat = rt['latitude'].max()
# bounds = [[max_lat, min_lng], [min_lat, max_lng]]

# # Create a base map centered around Australia
# middle_lat = (max_lat - min_lat) / 2
# middle_lng = (max_lng - min_lng) / 2
# map = folium.Map(location=[middle_lat, middle_lng],
#                  zoom_start=0)

# # Set the map boundaries to show only Australia
# map.fit_bounds(bounds)

# # Add the route lines
# folium.PolyLine(
#     locations=rt[['latitude', 'longitude']].values.tolist(),
#     color="green",
#     weight=4,
#     opacity=1
# ).add_to(map)

# # Add api sites
# for _, row in sites_df.iterrows():
#     folium.CircleMarker(
#         location=[row['latitude'], row['longitude']],
#         radius=2,
#         color="blue",
#         fill=True,
#         fill_color="blue",
#         fill_opacity=1,
#         tooltip=row['name']
#     ).add_to(map)

# # Add control stops
# for _, row in control_stops_df.iterrows():
#     folium.CircleMarker(
#         location=[row['latitude'], row['longitude']],
#         radius=3,
#         color="orange",
#         fill=True,
#         fill_color="orange",
#         fill_opacity=1,
#         tooltip=row['town'] + ': ' + row['location']
#     ).add_to(map)

# folium.Marker(
#                 location=[current_position['latitude'], current_position['longitude']],
#                 icon=folium.Icon(icon="car", prefix="fa")
#         ).add_to(map)

# map.options['maxBounds'] = bounds
# map.options['maxBoundsViscosity'] = 0.9
# map.options['minZoom'] = 5

# map


In [12]:
# from plotter import Plotter

# plotter = Plotter(route)
# plotter.add_api_sites(api_requester.get_current_sites)
# plotter.add_control_stops(route.get_control_stops_data)
# # plotter.add_current_position(current_position)
# current_position = {'longitude': 132.213897,
#                     'latitude': -14.425286}
# plotter.update_max_speed_distance(current_position)
# plotter.plot

# #TODO 
# #SE SI CHIAMA CLOUDMOVE NON ELIMINARE ALTRI DATI METEO MA UNIRLI
# #SE NON CI SONO DATI PLOTTARE NO DATA

In [13]:
#TODO i dati vengono sovrascritti e non aggiornati nei plot
# tenere ultimo SF e aggiornare dati di CM dove possibile
# se non ci sono dati di SF usare CM e viceversa

# se codice runna per tanto, i dropdown del tempo devono essere aggiornati

# per i weather plot bisogna correggere il cumDistance cosi che da 0 fino al primo cumDistance i valori siano 0 per tutti i tempi
# lo stesso vale dall'ultimo punto fino a route_data['cumDistance'].max()

# ratio item to take the mean or not of the data in the interval

# cambiare i font

import pandas as pd
import dash
from dash import Dash, html, dcc
from dash.dependencies import Input, Output, State

app = Dash(__name__)

# aCe images
url_image = 'https://avatars.githubusercontent.com/u/115261983?s=280&v=4'

## Time
# Slider
height_slider = 1000
unique_times = preprocessed_df.index.get_level_values('time').unique()
slider_marks = {i: time.strftime('%H:%M') for i, time in enumerate(unique_times)}

# Dropdown
unique_days = unique_times.map(lambda x: x.strftime('%d %B %Y')).unique().to_list()

# Update time
update_sec = 50

## Map
route_data = route.get_route_data
min_lng = route_data['longitude'].min()
min_lat = route_data['latitude'].min()
max_lng = route_data['longitude'].max()
max_lat = route_data['latitude'].max()
middle_lat = (max_lat + min_lat) / 2
middle_lng = (max_lng + min_lng) / 2

## Plot
x_cumDistance_km = route_data['cumDistance'] / 1000
x_cumDistance_str = 'Cumulative Distance (km)'

grid_color = 'lightgray'
box_color = 'black'

height_plot = 280

# Control stops shapes
def add_control_stops_lines(control_stops_df:pd.DataFrame) -> list:
        """ """
        # Save control stops dataframe without first and last row
        control_stops = control_stops_df.iloc[1:-1]

        # Loop over the control stops and create the vertical lines list
        vertical_lines = []
        for _, row in control_stops.iterrows():
            x_value = row['cumDistance'] / 1000
            vertical_lines.append(
                dict(type='line', x0=x_value, x1=x_value, y0=0, y1=1, yref='paper', line=dict(color='orange', dash='dash'))
            )
        
        return vertical_lines

app.layout = html.Div([
    # Header with image and title
    html.Div([
        # Title
        html.H1("Strategy Interface",
                style={'textAlign': 'left', 'margin': '0 auto', 'flex': '1'}),

        # Dropdown
        dcc.Dropdown(options=[{'label': day, 'value': day} for day in unique_days],
                     value=unique_days[0],
                     id='day-month-year-dropdown',
                     style={'width': '200px', 'margin-right': '20px'}),
        
        # Last update
        dcc.Interval(id='interval-update', interval=update_sec*1000, n_intervals=0),
        html.Div([
            html.Div(id='last-update-sf',
                     style={'color': 'gray'}),
            html.Div(id='last-update-cm',
                     style={'color': 'gray'}),
        ], style={'display': 'flex', 'flexDirection': 'column', 'margin-right': '20px'}),

        # Image
        html.Img(src=url_image,
                 style={'height': '100px', 'width': 'auto', 'margin-left': '20px'}),
        
    ], style={'display': 'flex', 'alignItems': 'center', 'padding-left': '20px', 'padding-right': '20px'}),


    # Container for the columns
    html.Div([
        # Map and static plots
        #TODO SPEGNEER 3D VIEW
        html.Div(children=[
            # Map
            dcc.Graph(
                id='map-plot',
                figure={
                    'data': [{
                        'type': 'scattermapbox',
                        'lat': route_data['latitude'],
                        'lon': route_data['longitude'],
                        'mode': 'lines+markers',
                        'marker': {'size': 3},
                        'line': {'color': 'green'},
                        'hoverinfo': 'none'
                    }],
                    'layout': {
                        'mapbox': {
                            'style': "open-street-map",
                            'center': {
                                'lat': middle_lat,
                                'lon': middle_lng
                            },
                            'zoom': 3,
                            'uirevision': True
                        },
                        'margin': {"r": 0, "t": 0, "l": 0, "b": 0}
                    }
                }
            ),

            # Max speed
            html.Br(),
            dcc.Graph(
                id='maxSpeed',
                figure={
                    'data': [
                        {
                            'x': x_cumDistance_km,
                            'y': route_data['maxSpeed'],
                            'type': 'scatter',
                            'mode': 'lines'
                        },
                    ],
                    'layout': {
                        'margin': {
                            "r": 0,
                            "t": 0
                            },
                        'xaxis': {
                            'title': x_cumDistance_str,
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'yaxis': {
                            'title': 'Max Speed (km/h)',
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'shapes': add_control_stops_lines(route.get_control_stops_data),
                        'height': height_plot
                    }
                }
            ),

            # Altitude
            html.Br(),
            dcc.Graph(
                id='altitude',
                figure={
                    'data': [
                        {
                            'x': x_cumDistance_km,
                            'y': route_data['altitudeSmoothed'],
                            'type': 'scatter',
                            'mode': 'lines'
                        },
                    ],
                    'layout': {
                        'margin': {
                            "r": 0,
                            "t": 0
                            },
                        'xaxis': {
                            'title': x_cumDistance_str,
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'yaxis': {
                            'title': 'Altitude (m)',
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'shapes': add_control_stops_lines(route.get_control_stops_data),
                        'height': height_plot
                    }
                }
            ),

            # Inclination
            html.Br(),
            dcc.Graph(
                id='inclination',
                figure={
                    'data': [
                        {
                            'x': x_cumDistance_km,
                            'y': route_data['inclinationSmoothed'],
                            'type': 'scatter',
                            'mode': 'lines'
                        },
                    ],
                    'layout': {
                        'margin': {
                            "r": 0,
                            "t": 0
                            },
                        'xaxis': {
                            'title': x_cumDistance_str,
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'yaxis': {
                            'title': 'Inclination (°)',
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'shapes': add_control_stops_lines(route.get_control_stops_data),
                        'height': height_plot
                    }
                }
            )
        ], style={'padding': 10, 'flex': 1}),

        # Time slider
        html.Div(
            dcc.RangeSlider(
                id='hour-minute-range-slider',
                min=0,
                max=len(unique_days) - 1,
                value=[0, len(unique_days) - 1],
                marks=slider_marks,
                vertical=True,
                step=1,
                verticalHeight=height_slider
            ),
            style={'width': '70px', 'height':height_slider},
            id='hour-minute-range-slider-container'
        ),

        # Weather plots
        html.Div(children=[
            dcc.Graph(id='plot-globalIrradiance'),
            dcc.Graph(id='plot-temperature'),
            dcc.Graph(id='plot-wind'),

        ], style={'padding': 10, 'flex': 1})
    ], style={'display': 'flex', 'flex-direction': 'row'}),
    
], style={'display': 'flex', 'flex-direction': 'column'})

## Callbacks
# Last update
@app.callback(
    Output('last-update-sf', 'children'),
    Output('last-update-cm', 'children'),
    Input('interval-update', 'n_intervals')
)
def update_output_div(n):
    time_SF_str = api_requester.get_solar_forecast_last_time
    if time_SF_str is pd.NaT:
        print_SF = "Last update SF: No data available"
    else:
        print_SF = f"Last update SF: {time_SF_str.strftime('%d %B %Y at %H:%M')}"

    time_CM_str = api_requester.get_solar_forecast_cloudmove_last_time
    if time_CM_str is pd.NaT:
        print_CM = "Last update CM: No data available"
    else:
        print_CM = f"Last update CM: {time_CM_str.strftime('%d %B %Y at %H:%M')}"
    return print_SF, print_CM

# Time callback
@app.callback(
    [Output('hour-minute-range-slider', 'max'),
     Output('hour-minute-range-slider', 'marks'),
     Output('hour-minute-range-slider', 'value'),
     Output('hour-minute-range-slider-container', 'style')],
     Input('day-month-year-dropdown', 'value')
)
def update_hour_minute_slider(day_month_year):
    unique_times_days = unique_times[unique_times.map(lambda x: x.strftime('%d %B %Y')) == day_month_year]
    slider_marks = {i: time.strftime('%H:%M') for i, time in enumerate(unique_times_days)}
    max = len(unique_times_days) - 1
    value = [0, max]
    style = {'width': '100px', 'height': f'{max*60}px'}
    return max, slider_marks, value, style

# Axis callback
@app.callback(
    [Output('maxSpeed', 'figure'),
     Output('altitude', 'figure'),
     Output('inclination', 'figure')],
    [Input('maxSpeed', 'relayoutData'),
     Input('altitude', 'relayoutData'),
     Input('inclination', 'relayoutData')],
    [State('maxSpeed', 'figure'),
     State('altitude', 'figure'),
     State('inclination', 'figure')]
)
def update_xaxis(maxSpeed_data, altitude_data, inclination_data, fig_max_speed, fig_altitude, fig_inclination):
    # Get the callback context to determine which input triggered the callback
    ctx = dash.callback_context
    if not ctx.triggered:
        raise dash.exceptions.PreventUpdate
    input_id = ctx.triggered[0]['prop_id'].split('.')[0]

    trigger_value = ctx.triggered[0]['value']
    # print(input_id)
    # print(trigger_value)
    # print(trigger_value.keys())
    # print("------------------")

    # Check if autorange is set for any of the graphs
    if 'autosize' in trigger_value and trigger_value['autosize']:
        if trigger_value['autosize']:
            raise dash.exceptions.PreventUpdate
    
    elif 'xaxis.autorange' in trigger_value and trigger_value['xaxis.autorange']:
        # If autorange is set for any graph, let all graphs determine their x-axis range automatically
        fig_max_speed['layout']['xaxis']['autorange'] = True
        fig_altitude['layout']['xaxis']['autorange'] = True
        fig_inclination['layout']['xaxis']['autorange'] = True

    else:
        # Determine the new x-axis range
        if input_id == 'maxSpeed':
            xaxis_range = [maxSpeed_data['xaxis.range[0]'], maxSpeed_data['xaxis.range[1]']]
        elif input_id == 'altitude':
            xaxis_range = [altitude_data['xaxis.range[0]'], altitude_data['xaxis.range[1]']]
        elif input_id == 'inclination':
            xaxis_range = [inclination_data['xaxis.range[0]'], inclination_data['xaxis.range[1]']]
        else:
            raise dash.exceptions.PreventUpdate

        # Update the x-axis range for all three figures and set autorange to False
        fig_max_speed['layout']['xaxis']['range'] = xaxis_range
        fig_max_speed['layout']['xaxis']['autorange'] = False

        fig_altitude['layout']['xaxis']['range'] = xaxis_range
        fig_altitude['layout']['xaxis']['autorange'] = False

        fig_inclination['layout']['xaxis']['range'] = xaxis_range
        fig_inclination['layout']['xaxis']['autorange'] = False

    return fig_max_speed, fig_altitude, fig_inclination

# Global irradiance
@app.callback(
    [Output('plot-globalIrradiance', 'figure'),
     Output('plot-temperature', 'figure'),
     Output('plot-wind', 'figure')],
    [Input('day-month-year-dropdown', 'value'),
    Input('hour-minute-range-slider', 'value'),
    Input('hour-minute-range-slider', 'marks')])
def update_graph(day_month_year, hour_minute, slider_marks):
    # Extract timezone from the DataFrame's index
    timezone = preprocessed_df.index.get_level_values('time').tz

    # Create the start and end timestamps for the day
    start_time = pd.Timestamp(day_month_year).tz_localize(timezone)
    end_time = start_time + pd.Timedelta(days=1)

    # Filter the DataFrame for the specific day
    data_for_selected_day = preprocessed_df[(preprocessed_df.index.get_level_values('time') >= start_time) & 
                            (preprocessed_df.index.get_level_values('time') < end_time)]
    
    # Create the start and end timestamps for the range
    first_datetime_str = day_month_year + " " + slider_marks[f'{hour_minute[0]}'] # Slider marks are str and hour_minute int
    first_datetime_obj = pd.Timestamp(first_datetime_str).tz_localize(timezone)
    last_datetime_str = day_month_year + " " + slider_marks[f'{hour_minute[1]}'] # Slider marks are str and hour_minute int
    last_datetime_obj = pd.Timestamp(last_datetime_str).tz_localize(timezone)

    # Create a mask to select the data in the range
    mask = (data_for_selected_day.index.get_level_values('time') >= first_datetime_obj) & (data_for_selected_day.index.get_level_values('time') <= last_datetime_obj)
    data_in_range_df = data_for_selected_day[mask]

    x_cumDistance_range_km = data_in_range_df.index.get_level_values('cumDistance').unique() / 1000

    figure_gh={
        'data': [{
            'x': x_cumDistance_range_km,
            'y': data_in_range_df['globalIrradiance'].groupby(level='cumDistance').mean(),
            'type': 'scatter',
            'mode': 'lines'}],
        'layout': {
            'margin': {"r": 0,
                       "t": 0},
            'xaxis': {
                'title': x_cumDistance_str,
                'showgrid': True,
                'gridcolor': grid_color,
                'showline': True,
                'linecolor': box_color},
            'yaxis': {
                'title': 'Global irradiance (W/m²)',
                'showgrid': True,
                'gridcolor': grid_color,
                'showline': True,
                'linecolor': box_color},
            'shapes': [{
                'type': 'line',
                'x0': x_cumDistance_range_km.min(),
                'x1': x_cumDistance_range_km.max(),
                'y0': data_for_selected_day['globalIrradiance'].max(),
                'y1': data_for_selected_day['globalIrradiance'].max(),
                'line': {
                    'color': 'red',
                    'width': 1}
                }
            ],
            'height': height_plot
        }
    }

    figure_tt={
        'data': [{
            'x': x_cumDistance_range_km,
            'y': data_in_range_df['temperature'].groupby(level='cumDistance').mean(),
            'type': 'scatter',
            'mode': 'lines'}],
        'layout': {
            'margin': {"r": 0,
                       "t": 0},
            'xaxis': {
                'title': x_cumDistance_str,
                'showgrid': True,
                'gridcolor': grid_color,
                'showline': True,
                'linecolor': box_color},
            'yaxis': {
                'title': 'Temperature (°C)',
                'showgrid': True,
                'gridcolor': grid_color,
                'showline': True,
                'linecolor': box_color},
            'shapes': [{
                'type': 'line',
                'x0': x_cumDistance_range_km.min(),
                'x1': x_cumDistance_range_km.max(),
                'y0': data_for_selected_day['temperature'].max(),
                'y1': data_for_selected_day['temperature'].max(),
                'line': {
                    'color': 'red',
                    'width': 1}
                },
                {
                'type': 'line',
                'x0': x_cumDistance_range_km.min(),
                'x1': x_cumDistance_range_km.max(),
                'y0': data_for_selected_day['temperature'].min(),
                'y1': data_for_selected_day['temperature'].min(),
                'line': {
                    'color': 'blue',
                    'width': 1}
                }
            ],
            'height': height_plot
        }
    }

    figure_ww={
        'data': [{
            'x': x_cumDistance_range_km,
            'y': data_in_range_df['windGust'].groupby(level='cumDistance').mean() * 3.6,
            'type': 'scatter',
            'mode': 'lines',
            'name': 'Wind gust'},
            {
            'x': x_cumDistance_range_km,
            'y': data_in_range_df['windSpeed'].groupby(level='cumDistance').mean() * 3.6,
            'type': 'scatter',
            'mode': 'lines',
            'name': 'Wind speed'},
            {
            'x': x_cumDistance_range_km,
            'y': data_in_range_df['sideWind'].groupby(level='cumDistance').mean() * 3.6,
            'type': 'scatter',
            'mode': 'lines',
            'name': 'Side wind'},
            {
            'x': x_cumDistance_range_km,
            'y': data_in_range_df['frontWind'].groupby(level='cumDistance').mean() * 3.6,
            'type': 'scatter',
            'mode': 'lines',
            'name': 'Front wind'},
            ],
        'layout': {
            'margin': {"r": 0,
                       "t": 0},
            'xaxis': {
                'title': x_cumDistance_str,
                'showgrid': True,
                'gridcolor': grid_color,
                'showline': True,
                'linecolor': box_color},
            'yaxis': {
                'title': 'Wind (km/h)',
                'showgrid': True,
                'gridcolor': grid_color,
                'showline': True,
                'linecolor': box_color},
            'height': height_plot,
            'legend': {
                'x': 0.2,
                'y': 1,
                'xanchor': 'left',
                'yanchor': 'top'
            }
        }
    }

    return figure_gh, figure_tt, figure_ww

if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode="tab")


Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>

In [14]:
import plotly.graph_objects as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import constants

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Graph(
        id='map',
        config={'displayModeBar': True, 'scrollZoom': True},
        figure={
            'data': [{
                'type': 'scattermapbox',
                'lat': [-25.2744],
                'lon': [133.7751],
                'mode': 'markers',
                'marker': {'size': 10},
            }],
            'layout': {
                'mapbox': {
                    'style': "mapbox://styles/mapbox/satellite-streets-v11",  # Satellite view
                    'center': {'lat': -25.2744, 'lon': 133.7751},
                    'zoom': 3,
                    'accesstoken': constants.TOKEN
                },
                'margin': {'l': 0, 'r': 0, 'b': 0, 't': 0}
            }
        }
    ),
    html.Pre(id='relayout-data', style={'border': '2px solid black'}),
])

@app.callback(
    Output('relayout-data', 'children'),
    Input('map', 'relayoutData')
)
def display_relayout_data(relayoutData):
    return str(relayoutData)

if __name__ == '__main__':
    app.run_server(debug=True)


AttributeError: module 'constants' has no attribute 'TOKEN'

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File ~\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\indexes\base.py:3652, in Index.get_loc(
    self=Index(['temperature', 'globalIrradiance', 'windS...'frontWind', 'airDensity'],
      dtype='object'),
    key='windGust'
)
   3651 try:
-> 3652     return self._engine.get_loc(casted_key)
        casted_key = 'windGust'
        self = Index(['temperature', 'globalIrradiance', 'windSpeed', 'windDirection',
       'sideWind', 'frontWind', 'airDensity'],
      dtype='object')
   3653 except KeyError as err:

File ~\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\_libs\index.pyx:147, in pandas._libs.index.IndexEngine.get_loc()

File ~\AppData\Local\Packages\PythonSoftwar